## Concatenate weather data obtained using NOAA API

- missing data in the EVAP dataset is filled by taking an average of the other years
- return an aggregated dataset with hierarchical indexes

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
cwd = os.getcwd()
datadir = '/'.join(cwd.split('/')[0:-1]) + '/data/'

combined_state_names = ['Alaska', 'Connecticut', 'Delaware', 'Maryland', 'Massachusetts', 
                   'Nevada', 'New Hampshire', 'New Mexico', 'Oklahoma', 'Rhode Island']
states = pd.read_csv(datadir+'external/'+'US_states.csv',index_col=0)
all_states = list(set(states.index) - set(combined_state_names) - {'District of Columbia'})

In [3]:
all_labels = ['TAVG','PRCP','SNOW','DP10','EVAP','AWND']
all_dfs = {}
for i in all_labels:
    all_dfs[i] = pd.read_csv(datadir+'external/'+i+'201501-201809.csv',index_col=0).iloc[:,1:-9]

In [4]:
a = pd.Series()
for i in all_dfs:
    dframe_i = all_dfs[i].groupby(pd.PeriodIndex(all_dfs[i].columns, freq='Y'), axis=1).mean()
    if i == 'EVAP': #filling NaNs in EVAP with yearly average
        dframe_i = dframe_i.T.fillna(dframe_i.mean(axis=1)).T
    combined_states = dframe_i.loc[combined_state_names,:].mean()
    dframe_i_processed = dframe_i.loc[list(all_states),:].copy()
    dframe_i_processed.loc['Other States',:] = combined_states
    concated = pd.concat([a,dframe_i_processed.stack().rename(i)],axis=1)
    a = concated
concated = concated.iloc[:,1:]

In [5]:
concated.index.levels

FrozenList([['Oregon', 'California', 'Illinois', 'Montana', 'Michigan', 'Wisconsin', 'Florida', 'Hawaii', 'Missouri', 'Mississippi', 'Indiana', 'New Jersey', 'Arizona', 'Colorado', 'West Virginia', 'Minnesota', 'Louisiana', 'Vermont', 'Kansas', 'Kentucky', 'Arkansas', 'Iowa', 'Maine', 'Tennessee', 'North Carolina', 'New York', 'Wyoming', 'North Dakota', 'Pennsylvania', 'Alabama', 'Ohio', 'Texas', 'Nebraska', 'South Carolina', 'Virginia', 'South Dakota', 'Washington', 'Utah', 'Idaho', 'Georgia', 'Other States'], [2015, 2016, 2017]])

In [6]:
concated.to_csv(datadir+'interim/'+'weather_2015-2017.csv')

In [8]:
concated.describe()

,TAVG,PRCP,SNOW,DP10,EVAP,AWND
count,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000
mean,54.249609,3.393369,2.367241,5.970446,6.752498,7.511230
std,8.614845,1.185794,2.173441,1.460268,6.976180,1.780203
min,41.650725,0.878576,0.000000,2.050376,2.050376,4.865138
25%,46.822516,2.611060,0.531528,5.047140,4.723922,6.085480
50%,53.175829,3.509808,1.992233,6.245793,6.083248,7.383034
75%,60.137129,4.122886,3.329974,6.976418,6.966594,8.704885
max,74.697925,6.008707,9.810074,8.930555,72.810000,13.659859


In [9]:
concated

TAVG      PRCP      SNOW      DP10      EVAP  \
Oregon         2015  50.685073  3.538181  0.979934  6.805906  7.977238   
               2016  49.058756  4.176896  1.611711  8.836773  8.018797   
               2017  48.252982  4.325047  2.630246  8.731434  8.731434   
California     2015  59.962052  1.268527  0.780166  2.383932  6.381611   
               2016  59.178019  2.302194  1.094336  3.537553  2.630887   
               2017  59.367843  2.618533  2.387031  3.593648  3.593648   
Illinois       2015  52.180614  3.843437  2.895723  6.362248  2.923000   
               2016  54.122068  3.262236  1.581601  5.893576  6.399013   
               2017  53.577671  3.254214  0.797189  5.560487  5.560487   
Montana        2015  44.432807  1.522242  3.796998  4.898928  6.453929   
               2016  43.945211  1.835551  3.254231  5.644297  5.875280   
               2017  42.613688  1.670185  5.488620  5.323421  5.323421   
Michigan       2015  45.119028  2.681948  4.904006  6.035568  4.488333   
               2016  47.450516  2.998342  7.371374  6.608891  6.681496   
               2017  46.384807  3.248662  5.585744  7.005287  7.005287   
Wisconsin      2015  45.494814  3.010905  2.857484  5.575086  6.762326   
               2016  47.011063  3.238361  4.034215  6.245459  6.983669   
               2017  45.742235  3.159028  2.836073  6.540983  6.540983   
Florida        2015  74.697925  4.535496  0.000000  6.737905  7.319167   
               2016  73.740461  4.581590  0.000000  6.221488  5.961552   
               2017  74.114473  5.070957  0.000000  6.268989  6.268989   
Hawaii         2015  72.004243  6.008707  0.000000  8.439617  3.770000   
               2016  72.054226  4.950410  0.000000  8.301226  9.639034   
               2017  71.850431  4.678549  0.000000  7.375969  7.375969   
Missouri       2015  55.615287  4.653167  0.852167  6.390156  4.749792   
               2016  57.206536  3.270899  0.328814  5.115103  5.561752   
               2017  56.828438  3.509808  0.212446  4.970400  4.970400   
Mississippi    2015  65.216262  5.129753  0.162452  6.713705  3.584275   
               2016  65.992152  4.633574  0.010942  5.848523  4.698052   
               2017  65.881472  5.238122  0.128888  6.832449  6.832449   
...                        ...       ...       ...       ...       ...   
Texas          2015  66.199357  4.080245  0.429544  4.922250  6.440392   
               2016  67.532748  3.396004  0.037859  4.273333  4.559968   
               2017  67.731417  3.018570  0.107273  3.766750  3.766750   
Nebraska       2015  50.785322  2.495035  1.864295  4.534400  6.564286   
               2016  51.653117  2.240331  1.938582  3.910884  4.609330   
               2017  50.754595  2.229462  1.917779  3.975230  3.975230   
South Carolina 2015  63.813182  5.210293  0.117045  7.068314  4.758611   
               2016  64.242110  3.716942  0.087008  5.203916  4.683439   
               2017  64.329080  4.165527  0.180357  5.941809  5.941809   
Virginia       2015  56.724423  4.057278  1.638717  7.088324  6.274064   
               2016  57.370200  3.868193  1.848930  6.465242  6.464880   
               2017  57.492613  3.502844  0.717874  6.083248  6.083248   
South Dakota   2015  47.464751  1.968298  2.882793  3.874580  5.610667   
               2016  48.272253  1.845905  2.837901  3.824252  4.294078   
               2017  46.723130  1.585006  2.566711  3.378191  3.378191   
Washington     2015  49.739745  3.861087  2.285630  6.950323  6.554381   
               2016  48.231012  4.245860  2.830868  8.930555  8.182753   
               2017  46.615206  4.197716  4.525078  8.324682  8.324682   
Utah           2015  47.054392  1.719302  3.152496  5.392786  8.469167   
               2016  46.631469  1.798910  3.746709  5.275192  5.100798   
               2017  48.465466  1.574260  4.212466  4.223255  4.223255   
Idaho          2015  46.278731  1.967131  2.925861  5.903788  3.484583   
               2016  45.028517